In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
cd '../gdrive/MyDrive/SSAC/4조'

/gdrive/.shortcut-targets-by-id/1ds5uus_7-EPK6G7A6HjwhTVmFOXL6YtQ/4조


In [ ]:
from gensim.models import Word2Vec
import os
import pandas as pd
from ast import literal_eval
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import multiprocessing
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 데이터 불러오기
movie_data = pd.read_csv('코드/Dataset/data_keyword.csv',low_memory=False)
movie_data.fillna('', inplace=True)

In [ ]:
movie_data.head(1)

,Unnamed: 0,id,cast_list,character_list,director,title,genres,overview,production_companies,release_date,keyword_list,keyword_list_no_space
0,0,862,TomHanks TimAllen DonRickles JimVarney Wallace...,Woody(voice) BuzzLightyear(voice) Mr.PotatoHea...,JohnLasseter,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",Pixar Animation Studios,1990,jealousy toy boy friendship friends rivalry bo...,jealousy toy boy friendship friends rivalry bo...
1,1,8844,RobinWilliams JonathanHyde KirstenDunst Bradle...,AlanParrish SamuelAlanParrish VanPelt JudyShep...,JoeJohnston,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...,TriStar Pictures Teitler Film Interscope Commu...,1990,board game disappearance new home recluse gian...,boardgame disappearance newhome recluse gianti...


In [ ]:
# feaetures에 들어있는 순서대로 단어들 corpus(말뭉치)에 합치기
features = ['keyword_list', 'genres', 'title', 'director', 'cast_list', 'character_list']
movie_data['corpus'] = ''
for feat in features:
    movie_data['corpus'] += " " + movie_data[feat]

In [ ]:
movie_data['corpus'][0]

' jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life Animation Comedy Family Toy Story JohnLasseter TomHanks TimAllen DonRickles JimVarney WallaceShawn Woody(voice) BuzzLightyear(voice) Mr.PotatoHead(voice) SlinkyDog(voice) Rex(voice)'

In [ ]:
# 모든 문자를 소문자로
movie_data['corpus'] = movie_data['corpus'].apply(lambda x : x.lower())

In [ ]:
movie_data['corpus'][0]

' jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life animation comedy family toy story johnlasseter tomhanks timallen donrickles jimvarney wallaceshawn woody(voice) buzzlightyear(voice) mr.potatohead(voice) slinkydog(voice) rex(voice)'

In [ ]:
# 정규표현식 적용 -> 특수문자 제거
import re
movie_data['corpus'] = movie_data['corpus'].apply(lambda x : re.sub("[^a-zA-Z]"," ",x))

In [ ]:
movie_data['corpus'][0]

' jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life animation comedy family toy story johnlasseter tomhanks timallen donrickles jimvarney wallaceshawn woody voice  buzzlightyear voice  mr potatohead voice  slinkydog voice  rex voice '

In [ ]:
# corpus(말뭉치)를 tokenize
def tokenize(data):
    temp = list(data)
    temp = [str.lower(i) for i in temp]
    temp = [list(i.split(' ')) for i in temp]

    return temp

tokenized_data = tokenize(movie_data["corpus"])

In [ ]:
tokenized_data[0]

['',
 'jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy',
 'next',
 'door',
 'new',
 'toy',
 'toy',
 'comes',
 'to',
 'life',
 'animation',
 'comedy',
 'family',
 'toy',
 'story',
 'johnlasseter',
 'tomhanks',
 'timallen',
 'donrickles',
 'jimvarney',
 'wallaceshawn',
 'woody',
 'voice',
 '',
 'buzzlightyear',
 'voice',
 '',
 'mr',
 'potatohead',
 'voice',
 '',
 'slinkydog',
 'voice',
 '',
 'rex',
 'voice',
 '']

In [ ]:
# 불용어 제거

# 불용어 목록 다운로드
nltk.download('stopwords')

def remove_stop_words(data):
    # 불용어(영어) 목록 불러오기
    stoplist=set(stopwords.words('english'))
    text=[]
    # 불용어 제거
    for i in data:
        text.append([word.replace('.', '') for word in i if word not in stoplist ])
    return text

tokenized_data = remove_stop_words(tokenized_data)

In [ ]:
tokenized_data[0]

['',
 'jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy',
 'next',
 'door',
 'new',
 'toy',
 'toy',
 'comes',
 'life',
 'animation',
 'comedy',
 'family',
 'toy',
 'story',
 'johnlasseter',
 'tomhanks',
 'timallen',
 'donrickles',
 'jimvarney',
 'wallaceshawn',
 'woody',
 'voice',
 '',
 'buzzlightyear',
 'voice',
 '',
 'mr',
 'potatohead',
 'voice',
 '',
 'slinkydog',
 'voice',
 '',
 'rex',
 'voice',
 '']

In [ ]:
# 공백 '' 제거
def remove_empty(data):
    text = []
    for i in data:
        text.append([word for word in i if word != ''])
    return text
tokenized_data = remove_empty(tokenized_data)

In [ ]:
tokenized_data[0]

['',
 'jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy',
 'next',
 'door',
 'new',
 'toy',
 'toy',
 'comes',
 'to',
 'life',
 'animation',
 'comedy',
 'family',
 'toy',
 'story',
 'johnlasseter',
 'tomhanks',
 'timallen',
 'donrickles',
 'jimvarney',
 'wallaceshawn',
 'woody',
 'voice',
 '',
 'buzzlightyear',
 'voice',
 '',
 'mr',
 'potatohead',
 'voice',
 '',
 'slinkydog',
 'voice',
 '',
 'rex',
 'voice',
 '']

In [ ]:
# Word2Vec 모델 정의
# window : window size, 주변 몇개 단어까지 볼지
# min_count : min_count 개수 이하로 등장한 단어는 사용 안함
# sg : skipgram을 사용한다는 뜻 (CBOW 말고)
# workers : multiprocess? 계산하는데 좋다는데 잘 모르는데 복붙함
def model(data):
    model_=Word2Vec(data, window=10, min_count=1, sg=1, workers = multiprocessing.cpu_count())
    return model_

In [ ]:
word2vec_model = model(tokenized_data)

In [ ]:
#word2vec_model.save('코드/model/good_w2v.model')

In [ ]:
movie_data.columns

Index(['Unnamed: 0', 'id', 'cast_list', 'character_list', 'director', 'title',
       'genres', 'overview', 'production_companies', 'release_date',
       'keyword_list', 'keyword_list_no_space', 'corpus'],
      dtype='object')

In [ ]:
# 추천 함수 정의
def recommend(search, model, topn=10):
    # 추천 목록 저장할 리스트 정의
    similar=[]
    # 검색하고 싶은 영화 제목을 형식에 맞게(공백제거, 소문자로) 변환
    search = search.replace(' ', '').lower()
    # 영화 데이터안에 있는 모든 영화들(title)에 대해서
    for title in movie_data['title'].apply(lambda x : x.replace(' ', '').lower()):
        similarity = 0
        try:
            # 검색하려는 영화(search)와 데이터 내의 영화(title)간의 유사도 계산
            similarity = model.wv.similarity(search, title)
            
        # 오류 무시
        except Exception as e:
            pass

        # 검색하려는 영화 자기 자신은 제와
        if title == search:
            continue
        
        # 각 영화의 제목, 유사도 저장
        similar.append((title,similarity))
        
    # 검색하려는 영화(search)와 다른 영화들 간의 유사도 데이터프레임 만들기            
    similar = list(set(similar))
    similar = pd.DataFrame(similar,columns=['title','score'])

    # feats = ['keyword_list', 'genres', 'director', 'cast_list', 'character_list']
    # similar[feats] = movie_data[feats]

    # 데이터프레임을 유사도순으로 정렬 후
    # 상위 topn(기본 10)개 만큼 출력
    return similar.sort_values(by='score',ascending=False)[:topn]['title']

In [ ]:
recommend('avengers', word2vec_model)

19235            hulk
24134        catwoman
25661    transformers
16409             cub
17542     dragonheart
24945       guardians
5875         warriors
69         millennium
2250        appleseed
5292         poseidon
Name: title, dtype: object

In [ ]:
recommend('zootopia', word2vec_model, topn=5)

4989        coraline
13523       barnyard
8518        raincoat
4917     ratatouille
21714         storks
Name: title, dtype: object

In [ ]:
recommend('ironman', word2vec_model, topn=5)

15623    captainamerica
897           poisonivy
3628         blackwidow
10912          zambezia
3484          galgameth
Name: title, dtype: object